<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Deepfake_and_Manipulated_Media_Analysis_R%26D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Deepfake Detection and Manipulated Media Analysis using Multiagent System and Compound AI Approach**

**Install Required Packages**

In [1]:
!pip install -q groq langchain langchain-community langgraph torch transformers opencv-python librosa numpy face-recognition dlib mediapipe scipy pillow tqdm pydantic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.2/138.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.3 MB/s eta 0:00:00


**Set Up the Groq Client**

In [6]:
import os
from groq import Groq
import getpass

# Prompt for the Groq API key and use it directly
GROQ_API_KEY = getpass.getpass("Enter your Groq API key: ")

# Initialize the Groq client
client = Groq(
    api_key=GROQ_API_KEY,
)

Enter your Groq API key: ··········


**Define Data Models**
- Define the data models for storing analysis results and reports:


In [7]:
from pydantic import BaseModel, Field
from datetime import datetime
from typing import Dict, List, Any, Optional, Union

class DeepfakeAnalysisResult(BaseModel):
    score: float = Field(..., description="Confidence score (0-1)")
    label: str = Field(..., description="Classification label")
    anomalies: List[str] = Field(default_factory=list)
    artifacts: List[str] = Field(default_factory=list)
    confidence: float = Field(..., description="Model confidence")
    method: str = Field(..., description="Detection method used")
    timestamp: datetime = Field(default_factory=datetime.now)

class MultimodalAnalysisReport(BaseModel):
    case_id: str
    file_info: Dict[str, Any]
    video_analysis: Optional[DeepfakeAnalysisResult]
    audio_analysis: Optional[DeepfakeAnalysisResult]
    image_analysis: Optional[DeepfakeAnalysisResult]
    text_analysis: Optional[DeepfakeAnalysisResult]
    multimodal_score: float
    verdict: str
    evidence: List[Dict[str, Any]]
    metadata: Dict[str, Any]

**Define Functions for Preprocessing**
- Define helper functions for preprocessing audio, image, and video data:

In [8]:
import librosa
import cv2
import numpy as np
import torch
from concurrent.futures import ThreadPoolExecutor

def reduce_noise(audio_data: np.ndarray) -> np.ndarray:
    return librosa.effects.preemphasis(audio_data)

def enhance_resolution(image: np.ndarray) -> np.ndarray:
    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    return sr.upsample(image)

def deblur_image(image: np.ndarray) -> np.ndarray:
    return cv2.filter2D(image, -1, np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]]))

def extract_audio_features(audio_data: np.ndarray, sample_rate: int) -> Dict[str, Any]:
    mfcc = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=13)
    chroma = librosa.feature.chroma_stft(y=audio_data, sr=sample_rate)
    mel = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
    return {"mfcc": mfcc, "chroma": chroma, "mel": mel}

def extract_image_features(image: np.ndarray) -> Dict[str, Any]:
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    return {"keypoints": keypoints, "descriptors": descriptors}

def calculate_dense_optical_flow(prev_frame: np.ndarray, curr_frame: np.ndarray) -> np.ndarray:
    gray_prev = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    gray_curr = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(gray_prev, gray_curr, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    return flow

def extract_temporal_features(optical_flow: np.ndarray) -> Dict[str, Any]:
    mag, ang = cv2.cartToPolar(optical_flow[..., 0], optical_flow[..., 1])
    return {"magnitude": mag, "angle": ang}

def estimate_noise(frame: np.ndarray) -> float:
    return np.mean(cv2.Laplacian(frame, cv2.CV_64F).var())

def calculate_contrast(frame: np.ndarray) -> float:
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return gray.std()

def detect_compression_artifacts(frame: np.ndarray) -> float:
    dct = cv2.dct(np.float32(frame) / 255.0)
    return np.mean(np.abs(dct))

**Define Preprocessing Functions**
- Define functions for preprocessing audio, image, and video data

In [9]:
import asyncio
import base64
import json
import re
from typing import Dict, Any

async def enhanced_preprocessing(file_path: str) -> Dict[str, Any]:
    file_extension = os.path.splitext(file_path)[1].lower()
    if file_extension in [".mp3", ".wav", ".flac"]:
        return await enhanced_audio_preprocessing(file_path)
    elif file_extension in [".jpg", ".jpeg", ".png", ".bmp"]:
        return await enhanced_image_preprocessing(file_path)
    elif file_extension in [".mp4", ".avi", ".mov", ".mkv"]:
        return await enhanced_video_preprocessing(file_path)
    else:
        raise ValueError("Unsupported file format")

async def enhanced_audio_preprocessing(audio_path: str) -> Dict[str, Any]:
    audio_data, sample_rate = librosa.load(audio_path, sr=None)
    audio_data = reduce_noise(audio_data)
    audio_features = extract_audio_features(audio_data, sample_rate)
    return {"audio": audio_data, "sample_rate": sample_rate, "audio_features": audio_features}

async def enhanced_image_preprocessing(image_path: str) -> Dict[str, Any]:
    image = cv2.imread(image_path)
    image = enhance_resolution(image)
    image = deblur_image(image)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_features = extract_image_features(image_rgb)
    return {"image": image_rgb, "image_features": image_features}

async def enhanced_video_preprocessing(video_path: str) -> Dict[str, Any]:
    cap = cv2.VideoCapture(video_path)
    frames = []
    audio_data = None
    metadata = {
        "fps": cap.get(cv2.CAP_PROP_FPS),
        "frame_count": int(cap.get(cv2.CAP_PROP_FRAME_COUNT)),
        "width": int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
        "height": int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
        "duration": float(cap.get(cv2.CAP_PROP_FRAME_COUNT)) / float(cap.get(cv2.CAP_PROP_FPS)),
        "codec": int(cap.get(cv2.CAP_PROP_FOURCC)).to_bytes(4, byteorder='little').decode(),
        "file_size": os.path.getsize(video_path)
    }
    frame_quality_metrics = []
    optical_flow_data = []
    prev_frame = None
    temporal_features = []

    loop = asyncio.get_event_loop()
    executor = ThreadPoolExecutor(max_workers=4)

    async def process_frame(frame):
        return await loop.run_in_executor(executor, enhance_resolution, frame)

    async def process_quality_metrics(frame):
        return await loop.run_in_executor(executor, lambda: {
            "blur": cv2.Laplacian(frame, cv2.CV_64F).var(),
            "noise": estimate_noise(frame),
            "brightness": np.mean(frame),
            "contrast": calculate_contrast(frame),
            "compression_artifacts": detect_compression_artifacts(frame)
        })

    async def process_optical_flow(prev_frame, frame):
        return await loop.run_in_executor(executor, calculate_dense_optical_flow, prev_frame, frame)

    async def process_temporal_features(flow):
        return await loop.run_in_executor(executor, extract_temporal_features, flow)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame, quality_metrics = await asyncio.gather(
            process_frame(frame),
            process_quality_metrics(frame)
        )
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(rgb_frame)
        frame_quality_metrics.append(quality_metrics)
        if prev_frame is not None:
            flow, temp_features = await asyncio.gather(
                process_optical_flow(prev_frame, frame),
                process_temporal_features(flow)
            )
            optical_flow_data.append(flow)
            temporal_features.append(temp_features)
        prev_frame = frame.copy()
    cap.release()

    try:
        video = VideoFileClip(video_path)
        audio = video.audio
        if audio is not None:
            audio_array = audio.to_soundarray()
            audio_data = extract_audio_features(audio_array, audio.fps)
        video.close()
    except Exception as e:
        print(f"Audio extraction error: {e}")
        audio_data = None

    return {
        "frames": frames,
        "audio": audio_data,
        "metadata": metadata,
        "quality_metrics": frame_quality_metrics,
        "optical_flow": optical_flow_data,
        "temporal_features": temporal_features
    }

**Define Functions for Analysis**
- functions for analyzing transcription, vision response, and other features

In [11]:
def analyze_transcription(transcription: str) -> List[str]:
    anomalies = []
    # Example: Check for repetitive phrases
    if "repeated phrase" in transcription.lower():
        anomalies.append("Repetitive phrases detected")
    # Example: Check for inconsistent timestamps
    if "inconsistent timestamp" in transcription.lower():
        anomalies.append("Inconsistent timestamps detected")
    return anomalies

def analyze_vision_response(response: str) -> List[str]:
    anomalies = []
    # Example: Check for blurry regions
    if "blurry region" in response.lower():
        anomalies.append("Blurry regions detected")
    # Example: Check for unnatural shadows
    if "unnatural shadow" in response.lower():
        anomalies.append("Unnatural shadows detected")
    return anomalies

def analyze_av_sync(frames: List[np.ndarray], audio: np.ndarray) -> float:
    # Example: Check for lip-sync consistency
    sync_score = 0.9  # Placeholder score
    return sync_score

def analyze_temporal_features(temporal_features: List[Dict[str, Any]]) -> List[str]:
    anomalies = []
    # Example: Check for abrupt changes in motion
    for feature in temporal_features:
        if feature["magnitude"].max() > 1.0:
            anomalies.append("Abrupt changes in motion detected")
    return anomalies

def analyze_optical_flow(optical_flow: List[np.ndarray]) -> List[str]:
    anomalies = []
    # Example: Check for inconsistent flow patterns
    for flow in optical_flow:
        if flow.max() > 1.0:
            anomalies.append("Inconsistent flow patterns detected")
    return anomalies

def analyze_biometric_consistency(frames: List[np.ndarray], models: Dict[str, Any]) -> float:
    # Example: Check for consistent facial landmarks
    consistency_score = 0.9  # Placeholder score
    return consistency_score

**Define Analysis Functions**
- Define functions for analyzing audio, image, and video data using Groq models

In [12]:
async def advanced_audio_analysis(audio_data: Dict[str, Any], models: Dict[str, Any], device: torch.device) -> DeepfakeAnalysisResult:
    if audio_data is None:
        return DeepfakeAnalysisResult(
            score=0.0,
            label="NO_AUDIO",
            confidence=0.0,
            method="audio_analysis",
            anomalies=["No audio data available"]
        )
    anomalies = []
    scores = []

    # Use Groq's speech-to-text model
    response = client.audio.transcriptions.create(
        file=audio_data["waveform"],
        model="whisper-large-v3-turbo",
    )
    transcription = response.text

    # Analyze the transcription for anomalies
    anomalies.extend(analyze_transcription(transcription))
    scores.append(0.9)  # Placeholder score, adjust based on your analysis

    final_score = np.mean(scores)
    return DeepfakeAnalysisResult(
        score=float(final_score),
        label="REAL" if final_score > 0.7 else "FAKE",
        confidence=float(np.std(scores)),
        method="audio_analysis",
        anomalies=anomalies
    )

async def advanced_image_analysis(image: np.ndarray, models: Dict[str, Any], device: torch.device) -> DeepfakeAnalysisResult:
    anomalies = []
    scores = []

    # Use Groq's vision model
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"Analyze this image for signs of digital manipulation or inconsistencies.",
                "image_url": "data:image/png;base64," + base64.b64encode(cv2.imencode('.png', image)[1]).decode()
            }
        ],
        model="llama-3.2-90b-vision-preview",
    )
    content = response.choices[0].message.content
    score_match = re.search(r"Score: (0\.\d+|1\.0)", content)
    if score_match:
        scores.append(float(score_match.group(1)))
    anomalies.extend(analyze_vision_response(content))

    final_score = np.mean(scores)
    return DeepfakeAnalysisResult(
        score=float(final_score),
        label="REAL" if final_score > 0.7 else "FAKE",
        confidence=float(np.std(scores)),
        method="image_analysis",
        anomalies=anomalies
    )

async def advanced_video_analysis(video_data: Dict[str, Any], models: Dict[str, Any], device: torch.device) -> DeepfakeAnalysisResult:
    anomalies = []
    scores = []
    videomae_model = models["videomae"].to(device)
    videomae_processor = VideoMAEFeatureExtractor.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics")
    inputs = videomae_processor(video_data["frames"], return_tensors="pt").to(device)
    with torch.no_grad():
        videomae_output = videomae_model(**inputs)
        videomae_score = torch.softmax(videomae_output.logits, dim=-1)
        scores.append(videomae_score.max().item())
    timesformer_model = models["timesformer"].to(device)
    timesformer_processor = VideoMAEFeatureExtractor.from_pretrained("facebook/timesformer-base-finetuned-k400")
    inputs = timesformer_processor(video_data["frames"], return_tensors="pt").to(device)
    with torch.no_grad():
        timesformer_output = timesformer_model(**inputs)
        timesformer_score = torch.softmax(timesformer_output.logits, dim=-1)
        scores.append(timesformer_score.max().item())
    llava_model = models["llava"].to(device)
    llava_processor = models["processors"]["llava"]
    prompt = "Analyze this video for signs of digital manipulation or inconsistencies."
    inputs = llava_processor(video=video_data["frames"], text=prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = llava_model.generate(**inputs)
        analysis = llava_processor.decode(outputs[0], skip_special_tokens=True)
        llava_score = analyze_llava_response(analysis)
        scores.append(llava_score)
    temporal_anomalies = analyze_temporal_features(video_data["temporal_features"])
    anomalies.extend(temporal_anomalies)
    optical_flow_anomalies = analyze_optical_flow(video_data["optical_flow"])
    anomalies.extend(optical_flow_anomalies)
    final_score = np.mean(scores)
    return DeepfakeAnalysisResult(
        score=float(final_score),
        label="REAL" if final_score > 0.7 else "FAKE",
        confidence=float(np.std(scores)),
        method="video_analysis",
        anomalies=anomalies
    )

**Define Tool Creation Functions**
- Define functions for creating deepfake detection tools and agents

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain.chains import LLMChain
from langchain.tools import BaseTool
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent
from langchain.schema import AgentAction, AgentFinish
from langgraph.graph import Graph, StateGraph, END

def create_deepfake_detection_tools(models: Dict[str, Any], device: torch.device) -> List[Tool]:
    tools = [
        Tool(
            name="analyze_video",
            func=lambda x: advanced_video_analysis(x, models, device),
            description="Analyzes video content for signs of manipulation"
        ),
        Tool(
            name="analyze_audio",
            func=lambda x: advanced_audio_analysis(x, models, device),
            description="Analyzes audio content for signs of manipulation"
        ),
        Tool(
            name="analyze_image",
            func=lambda x: advanced_image_analysis(x, models, device),
            description="Analyzes image content for signs of manipulation"
        ),
        Tool(
            name="semantic_analysis",
            func=lambda x: semantic_consistency_analysis(x, models["llms"]),
            description="Analyzes semantic consistency across modalities"
        ),
        Tool(
            name="face_forgery_detection",
            func=lambda x: face_forgery_detection(x, models, device),
            description="Detects face forgeries in video content"
        ),
        Tool(
            name="lip_sync_detection",
            func=lambda x: lip_sync_detection(x, models, device),
            description="Analyzes lip-sync consistency between audio and video"
        ),
        Tool(
            name="background_consistency",
            func=lambda x: background_consistency_analysis(x, models, device),
            description="Analyzes background consistency across frames"
        ),
        Tool(
            name="real_time_streaming_analysis",
            func=lambda x: real_time_streaming_analysis(x, models, device),
            description="Analyzes live video streams for signs of manipulation"
        ),
        Tool(
            name="text_analysis",
            func=lambda x: text_analysis(x, models["llms"]),
            description="Analyzes text content for signs of manipulation"
        ),
        Tool(
            name="metadata_analysis",
            func=lambda x: metadata_analysis(x),
            description="Analyzes metadata for signs of manipulation"
        )
    ]
    return tools

def create_detection_agent(tools: List[Tool], llm: ChatOpenAI) -> AgentExecutor:
    class DeepfakeDetectionOutputParser:
        def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
            try:
                if "Final Answer:" in llm_output:
                    return AgentFinish(
                        return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                        log=llm_output,
                    )
                action_match = re.search(r"Action: (.*?)\nAction Input: (.*)", llm_output, re.DOTALL)
                if not action_match:
                    raise ValueError("Could not parse LLM output: " + llm_output)
                action = action_match.group(1).strip()
                action_input = action_match.group(2).strip()
                return AgentAction(tool=action, tool_input=action_input, log=llm_output)
            except Exception as e:
                raise ValueError(f"Could not parse LLM output: {llm_output}") from e

    prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template("""
            You are an expert deepfake detection system. Your goal is to analyze content across multiple modalities
            to determine authenticity. Consider all available evidence and patterns including:

            1. Visual elements: inconsistencies, artifacts, unnatural patterns
            2. Audio characteristics: synthetic artifacts, unnatural transitions
            3. Semantic coherence: logical consistency across modalities
            4. Temporal patterns: synchronization, continuity
            5. Biometric features: facial landmarks, expressions, movements

            Available tools:
            {tools}

            Process:
            1. Analyze the input using appropriate tools
            2. Evaluate evidence across modalities
            3. Make a final determination on authenticity

            Format your response as:
            Action: [tool name]
            Action Input: [tool input]
            Observation: [result]
            ... (repeat for additional tools as needed)
            Final Answer: [detailed analysis and verdict]
            """),
        HumanMessagePromptTemplate.from_template("{input}")
    ])

    return AgentExecutor.from_agent_and_tools(
        agent=LLMSingleActionAgent(
            llm_chain=LLMChain(llm=llm, prompt=prompt),
            output_parser=DeepfakeDetectionOutputParser(),
            stop=["Observation:", "Final Answer:"],
            allowed_tools=[tool.name for tool in tools]
        ),
        tools=tools,
        verbose=True
    )

**Define Detection Graph**
- Define the detection graph for processing the input data

In [14]:
def create_detection_graph() -> StateGraph:
    def preprocess(state):
        input_data = state["input"]
        processed_data = asyncio.run(enhanced_preprocessing(input_data))
        return {**state, "processed_data": processed_data}

    def analyze_modalities(state):
        processed_data = state["processed_data"]
        models = state["models"]
        device = state["device"]
        results = {
            "video": asyncio.run(advanced_video_analysis(processed_data, models, device)) if "frames" in processed_data else None,
            "audio": asyncio.run(advanced_audio_analysis(processed_data.get("audio"), models, device)) if "audio" in processed_data else None,
            "image": asyncio.run(advanced_image_analysis(processed_data["image"], models, device)) if "image" in processed_data else None,
            "text": asyncio.run(text_analysis(processed_data.get("text"), models["llms"])) if "text" in processed_data else None
        }
        return {**state, "modality_results": results}

    def cross_modal_analysis(state):
        results = state["modality_results"]
        processed_data = state["processed_data"]
        models = state["models"]
        cross_modal_score = analyze_cross_modal_consistency(results, processed_data, models)
        return {**state, "cross_modal_score": cross_modal_score}

    def generate_report(state):
        results = state["modality_results"]
        cross_modal_score = state["cross_modal_score"]
        processed_data = state["processed_data"]
        report = generate_comprehensive_report(results, cross_modal_score, processed_data)
        return {**state, "final_report": report}

    workflow = StateGraph(nodes=[
        ("preprocess", preprocess),
        ("analyze_modalities", analyze_modalities),
        ("cross_modal_analysis", cross_modal_analysis),
        ("generate_report", generate_report)
    ])
    workflow.add_edge("preprocess", "analyze_modalities")
    workflow.add_edge("analyze_modalities", "cross_modal_analysis")
    workflow.add_edge("cross_modal_analysis", "generate_report")
    workflow.add_edge("generate_report", END)
    return workflow

**Define Functions for Analyzing Cross-Modal Consistency and Generating Reports**

In [15]:
def analyze_cross_modal_consistency(
    results: Dict[str, DeepfakeAnalysisResult],
    processed_data: Dict[str, Any],
    models: Dict[str, Any]
) -> float:
    scores = []
    if results["audio"] and results["video"]:
        sync_score = analyze_av_sync(processed_data["frames"], processed_data["audio"])
        scores.append(sync_score)
    semantic_score = analyze_semantic_consistency(results, processed_data, models["llms"])
    scores.append(semantic_score)
    temporal_score = analyze_temporal_coherence(processed_data["temporal_features"])
    scores.append(temporal_score)
    bio_score = analyze_biometric_consistency(processed_data["frames"], models)
    scores.append(bio_score)
    return float(np.mean(scores))

def analyze_semantic_consistency(
    results: Dict[str, DeepfakeAnalysisResult],
    processed_data: Dict[str, Any],
    llms: Dict[str, ChatOpenAI]
) -> float:
    prompt = ChatPromptTemplate.from_template("""
        Analyze the consistency between different modalities in the content:

        Video Analysis: {video_analysis}
        Audio Analysis: {audio_analysis}
        Image Analysis: {image_analysis}
        Text Analysis: {text_analysis}

        Consider:
        1. Do the modalities tell a coherent story?
        2. Are there logical contradictions?
        3. Do temporal patterns align?
        4. Is the emotional content consistent?

        Rate the consistency from 0 to 1, where 1 is perfectly consistent.
        Provide detailed reasoning for your rating.

        Output format:
        Score: [0-1]
        Reasoning: [detailed explanation]
    """)
    chain = LLMChain(llm=llms["gpt4"], prompt=prompt)
    response = chain.run({
        "video_analysis": results["video"].dict() if results["video"] else "N/A",
        "audio_analysis": results["audio"].dict() if results["audio"] else "N/A",
        "image_analysis": results["image"].dict() if results["image"] else "N/A",
        "text_analysis": results["text"].dict() if results["text"] else "N/A"
    })
    score_match = re.search(r"Score: (0\.\d+|1\.0)", response)
    if score_match:
        return float(score_match.group(1))
    return 0.5

def generate_comprehensive_report(
    results: Dict[str, DeepfakeAnalysisResult],
    cross_modal_score: float,
    processed_data: Dict[str, Any]
) -> MultimodalAnalysisReport:
    scores = [
        results["video"].score if results["video"] else 0.5,
        results["audio"].score if results["audio"] else 0.5,
        results["image"].score if results["image"] else 0.5,
        results["text"].score if results["text"] else 0.5,
        cross_modal_score
    ]
    weights = [0.3, 0.2, 0.2, 0.2, 0.1]
    final_score = sum(s * w for s, w in zip(scores, weights))
    evidence = []
    for modality, result in results.items():
        if result:
            evidence.extend([
                {
                    "type": modality,
                    "description": anomaly,
                    "confidence": result.confidence,
                    "method": result.method
                }
                for anomaly in result.anomalies
            ])
    return MultimodalAnalysisReport(
        case_id=f"DFD-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
        file_info=processed_data["metadata"] if "metadata" in processed_data else {},
        video_analysis=results["video"],
        audio_analysis=results["audio"],
        image_analysis=results["image"],
        text_analysis=results["text"],
        multimodal_score=float(final_score),
        verdict="AUTHENTIC" if final_score > 0.7 else "MANIPULATED",
        evidence=evidence,
        metadata={
            "processing_time": datetime.now().isoformat(),
            "models_used": list(results.keys()),
            "cross_modal_score": cross_modal_score,
            "confidence_distribution": {
                modality: result.confidence
                for modality, result in results.items() if result
            }
        }
    )

**Define Real-Time Streaming Analysis**
- Define a function for real-time streaming analysis

In [16]:
async def real_time_streaming_analysis(video_stream: Any, models: Dict[str, Any], device: torch.device) -> DeepfakeAnalysisResult:
    frames = []
    audio_data = None
    metadata = {
        "fps": 30,  # Assuming 30 FPS for real-time streaming
        "frame_count": 0,
        "width": 0,
        "height": 0,
        "duration": 0,
        "codec": "N/A",
        "file_size": 0
    }
    frame_quality_metrics = []
    optical_flow_data = []
    prev_frame = None
    temporal_features = []

    loop = asyncio.get_event_loop()
    executor = ThreadPoolExecutor(max_workers=4)

    async def process_frame(frame):
        return await loop.run_in_executor(executor, enhance_resolution, frame)

    async def process_quality_metrics(frame):
        return await loop.run_in_executor(executor, lambda: {
            "blur": cv2.Laplacian(frame, cv2.CV_64F).var(),
            "noise": estimate_noise(frame),
            "brightness": np.mean(frame),
            "contrast": calculate_contrast(frame),
            "compression_artifacts": detect_compression_artifacts(frame)
        })

    async def process_optical_flow(prev_frame, frame):
        return await loop.run_in_executor(executor, calculate_dense_optical_flow, prev_frame, frame)

    async def process_temporal_features(flow):
        return await loop.run_in_executor(executor, extract_temporal_features, flow)

    while True:
        frame = await video_stream.read()
        if frame is None:
            break
        frame, quality_metrics = await asyncio.gather(
            process_frame(frame),
            process_quality_metrics(frame)
        )
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(rgb_frame)
        frame_quality_metrics.append(quality_metrics)
        if prev_frame is not None:
            flow, temp_features = await asyncio.gather(
                process_optical_flow(prev_frame, frame),
                process_temporal_features(flow)
            )
            optical_flow_data.append(flow)
            temporal_features.append(temp_features)
        prev_frame = frame.copy()

    video_data = {
        "frames": frames,
        "audio": audio_data,
        "metadata": metadata,
        "quality_metrics": frame_quality_metrics,
        "optical_flow": optical_flow_data,
        "temporal_features": temporal_features
    }

    return await advanced_video_analysis(video_data, models, device)

**Define Text Analysis**
- Define a function for text analysis using Groq models:

In [17]:
async def text_analysis(text: str, llms: Dict[str, ChatOpenAI]) -> DeepfakeAnalysisResult:
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"Analyze the following text for signs of manipulation or inconsistencies:\n\n{text}"
            }
        ],
        model="llama-3.3-70b-versatile",
    )
    content = response.choices[0].message.content
    score_match = re.search(r"Score: (0\.\d+|1\.0)", content)
    if score_match:
        score = float(score_match.group(1))
    else:
        score = 0.5
    anomalies = [line.strip() for line in content.split("Reasoning:")[-1].strip().split("\n") if line.strip()]
    return DeepfakeAnalysisResult(
        score=score,
        label="REAL" if score > 0.7 else "FAKE",
        confidence=0.0,
        method="text_analysis",
        anomalies=anomalies
    )

**Define Metadata Analysis**
- Define a function for metadata analysis

In [18]:
def metadata_analysis(metadata: Dict[str, Any]) -> DeepfakeAnalysisResult:
    anomalies = []
    scores = []

    # Example metadata analysis
    if metadata.get("fps") < 10:
        anomalies.append("Unusually low frame rate")
        scores.append(0.2)
    if metadata.get("duration") < 1:
        anomalies.append("Unusually short duration")
        scores.append(0.2)
    if metadata.get("file_size") < 100000:
        anomalies.append("Unusually small file size")
        scores.append(0.2)

    final_score = np.mean(scores) if scores else 0.5
    return DeepfakeAnalysisResult(
        score=float(final_score),
        label="REAL" if final_score > 0.7 else "FAKE",
        confidence=float(np.std(scores)) if scores else 0.0,
        method="metadata_analysis",
        anomalies=anomalies
    )

**Define the Environment Setup**
- Define the function to set up the enhanced detection environment

In [20]:
def setup_enhanced_detection_environment():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    models = {
        "videomae": VideoMAEForVideoClassification.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics"),
        "timesformer": TimesformerForVideoClassification.from_pretrained("facebook/timesformer-base-finetuned-k400"),
        "llava": LlavaForConditionalGeneration.from_pretrained("llava/llava-v1"),
        "clip": CLIPModel.from_pretrained("openai/clip-vit-large-patch14"),
        "instructblip": InstructBlipForConditionalGeneration.from_pretrained("Salesforce/instructblip-vicuna-7b"),
        "vit_gpt2": VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning"),
        "efficientnet": EfficientNetForImageClassification.from_pretrained("google/efficientnet-b7"),
        "wav2vec2": Wav2Vec2ForSequenceClassification.from_pretrained("facebook/wav2vec2-large-960h"),
        "whisper": WhisperForAudioClassification.from_pretrained("openai/whisper-large-v3"),
        "hubert": HubertForCTC.from_pretrained("facebook/hubert-large-ls960-ft"),
        "llama3": LlamaForCausalLM.from_pretrained("meta-llama/Llama-30b"),
        "llama3_1": LlamaForCausalLM.from_pretrained("meta-llama/Llama-33b"),
        "roberta_fake": AutoModelForSequenceClassification.from_pretrained("deepset/roberta-base-squad2"),
        "blenderbot": BlenderbotForConditionalGeneration.from_pretrained("facebook/blenderbot-400M-distill"),
        "bert": BertForSequenceClassification.from_pretrained("bert-base-uncased"),
        "t5": T5ForConditionalGeneration.from_pretrained("t5-base"),
        "face_detector": mp.solutions.face_detection.FaceDetection(min_detection_confidence=0.7),
        "face_mesh": mp.solutions.face_mesh.FaceMesh(
            static_image_mode=False,
            max_num_faces=1,
            min_detection_confidence=0.7
        ),
        "processors": {
            "clip": CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14"),
            "instructblip": InstructBlipProcessor.from_pretrained("Salesforce/instructblip-vicuna-7b"),
            "vit": ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k"),
            "wav2vec2": Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h"),
            "whisper": WhisperProcessor.from_pretrained("openai/whisper-large-v3"),
            "roberta": AutoTokenizer.from_pretrained("deepset/roberta-base-squad2"),
            "blenderbot": BlenderbotTokenizer.from_pretrained("facebook/blenderbot-400M-distill"),
            "bert": BertTokenizer.from_pretrained("bert-base-uncased"),
            "t5": T5Tokenizer.from_pretrained("t5-base"),
            "efficientnet": AutoImageProcessor.from_pretrained("google/efficientnet-b7"),
            "hubert": Wav2Vec2Processor.from_pretrained("facebook/hubert-large-ls960-ft"),
            "llava": LlavaProcessor.from_pretrained("llava/llava-v1"),
            "llama3": LlamaTokenizer.from_pretrained("meta-llama/Llama-30b"),
            "llama3_1": LlamaTokenizer.from_pretrained("meta-llama/Llama-33b")
        },
        "llms": {
            "gpt4": ChatOpenAI(model="gpt-4", temperature=0.2),
            "claude": ChatOpenAI(model="claude-3-opus", temperature=0.2)
        }
    }
    return {"device": device, "models": models}

**Define the Main Function**
- Define the main function to run the deepfake detection system

In [21]:
def run_deepfake_detection(file_path: str, mode: str = "all") -> MultimodalAnalysisReport:
    try:
        env = setup_enhanced_detection_environment()
        tools = create_deepfake_detection_tools(env["models"], env["device"])
        agent = create_detection_agent(tools, env["models"]["llms"]["gpt4"])
        workflow = create_detection_graph()
        initial_state = {
            "input": file_path,
            "mode": mode,
            "models": env["models"],
            "device": env["device"]
        }
        final_state = workflow.run(initial_state)
        return final_state["final_report"]
    except Exception as e:
        print(f"Error in deepfake detection: {str(e)}")
        raise

if __name__ == "__main__":
    file_path = "actual/path/to/your/content.mp4"
    report = asyncio.run(run_deepfake_detection(file_path))
    print("\nDeepfake Detection Report:")
    print(json.dumps(report.dict(), indent=2, default=str))

Error in deepfake detection: name 'VideoMAEForVideoClassification' is not defined


NameError: name 'VideoMAEForVideoClassification' is not defined